# Phase 5: P.1812 Loss Calculation

**Phase 5**: Process Phase 4 CSV profiles through ITU-R P.1812-6 model and calculate radio propagation loss.

This notebook:
- Loads CSV profiles from Phase 4 (already P.1812-ready with transmitter point at d=0)
- Processes each profile using P.1812 bt_loss() function
- Calculates basic transmission loss (Lb) and electric field strength (Ep)
- Exports results to structured DataFrame and CSV file

## Prerequisites
- Phase 0: Setup (defines CONFIG, project_root, data directories)
- Phase 1-4: Must be completed to generate P.1812-ready CSV profiles

## Output
- CSV file: `data/output/spreadsheets/p1812_results.csv`
- Summary statistics and visualizations

## 1. Setup from Phase 0

In [ ]:
%run phase0_setup.ipynb

## 2. Additional Imports

In [ ]:
import ast
import csv
import time
from typing import List, Tuple

# P.1812-6 propagation model
import Py1812.P1812
import matplotlib.pyplot as plt

print("✓ Phase 5 imports successful")

## 3. Setup Output Path

In [ ]:
# Output directory for results
output_dir = project_root / 'data' / 'output' / 'spreadsheets'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Input profiles dir: {profiles_dir}")
print(f"Output results dir: {output_dir}")

# Verify Phase 4 profiles exist
csv_files = list(profiles_dir.glob("*.csv"))
print(f"\nFound {len(csv_files)} CSV profile file(s)")
if csv_files:
    for f in csv_files:
        print(f"  - {f.name}")

## 4. Load CSV Profiles

In [ ]:
def load_profiles(profiles_dir: Path) -> List[List[str]]:
    """
    Load all CSV profiles from directory.
    Skip header rows.
    """
    profiles = []
    
    csv_files = list(profiles_dir.glob("*.csv"))
    print(f"Loading {len(csv_files)} profile file(s)...")
    
    for file in csv_files:
        print(f"  Loading: {file.name}")
        with file.open(newline="", encoding="utf-8") as f:
            # Skip header (first line), read remaining rows
            rows = list(csv.reader(f, delimiter=";"))[1:]
            profiles.extend(rows)
            print(f"    -> {len(rows)} profiles")
    
    print(f"\nTotal profiles: {len(profiles)}")
    return profiles

# Load all profiles
profiles = load_profiles(profiles_dir)

if not profiles:
    print("\n⚠ WARNING: No profiles found!")
    print("Make sure Phase 4 has been executed to generate CSV profiles.")

## 5. Parse Profile Parameters

In [ ]:
def process_loss_parameters(profile: List[str]) -> Tuple:
    """
    Parse CSV row into P.1812 bt_loss() parameters.
    
    CSV columns (first 14 used, skip azimuth at index 14):
    0: f (frequency)
    1: p (time percentage)
    2: d (distance array - starts at 0, added by Phase 4)
    3: h (height array)
    4: R (resistance array)
    5: Ct (land cover category array)
    6: zone (zone ID array)
    7: htg (TX antenna height)
    8: hrg (RX antenna height)
    9: pol (polarization)
    10: phi_t (TX latitude)
    11: phi_r (RX latitude)
    12: lam_t (TX longitude)
    13: lam_r (RX longitude)
    14: azimuth (skip)
    """
    # Parse first 14 columns
    parameters = [ast.literal_eval(parameter) for parameter in profile[0:14]]
    
    return (
        float(parameters[0]),                                  # f: frequency (GHz)
        float(parameters[1]),                                  # p: time percentage (%)
        np.array([float(v) for v in parameters[2]]),          # d: distances (km)
        np.array([float(v) for v in parameters[3]]),          # h: heights (m)
        np.array([float(v) for v in parameters[4]]),          # R: resistances (ohms)
        np.array([int(v) for v in parameters[5]]),            # Ct: land cover categories
        np.array([int(v) for v in parameters[6]]),            # zone: zone IDs
        float(parameters[7]),                                  # htg: TX antenna height (m)
        float(parameters[8]),                                  # hrg: RX antenna height (m)
        int(parameters[9]),                                    # pol: polarization
        float(parameters[10]),                                 # phi_t: TX latitude
        float(parameters[11]),                                 # phi_r: RX latitude
        float(parameters[12]),                                 # lam_t: TX longitude
        float(parameters[13]),                                 # lam_r: RX longitude
    )

# Test on first profile
if profiles:
    params = process_loss_parameters(profiles[0])
    print(f"Profile 1 parameters:")
    print(f"  Frequency: {params[0]} GHz")
    print(f"  Time %: {params[1]}%")
    print(f"  Distance points: {len(params[2])}")
    print(f"  First distance (TX point): {params[2][0]} km")
    print(f"  Last distance (RX point): {params[2][-1]:.2f} km")
    print(f"  TX location: lat={params[10]:.4f}, lon={params[12]:.4f}")

## 6. Execute P.1812 Loss Calculation

In [ ]:
def calculate_p1812_loss(parameters: Tuple) -> Tuple[float, float]:
    """
    Call P.1812 bt_loss() function.
    
    Returns (Lb, Ep) where:
    - Lb: Basic transmission loss (dB)
    - Ep: Electric field strength (dBμV/m)
    """
    try:
        Lb, Ep = Py1812.P1812.bt_loss(*parameters)
        return float(Lb), float(Ep)
    except Exception as e:
        print(f"  ⚠ Error: {e}")
        return np.nan, np.nan

# Test on first profile
if profiles:
    print("Testing P.1812 calculation...")
    params = process_loss_parameters(profiles[0])
    start = time.perf_counter()
    Lb, Ep = calculate_p1812_loss(params)
    elapsed = time.perf_counter() - start
    
    if not np.isnan(Lb):
        print(f"✓ Success:")
        print(f"  Lb: {Lb:.2f} dB")
        print(f"  Ep: {Ep:.2f} dBμV/m")
    else:
        print(f"✗ Failed (NaN result)")
    print(f"  Time: {elapsed:.4f} seconds")

## 7. Process All Profiles

In [ ]:
def process_all_profiles(profiles: List[List[str]]) -> pd.DataFrame:
    """
    Process all profiles through P.1812.
    """
    results = []
    failed = 0
    start_total = time.perf_counter()
    
    for idx, profile in enumerate(profiles):
        try:
            params = process_loss_parameters(profile)
            Lb, Ep = calculate_p1812_loss(params)
            
            f, p, d, h, R, Ct, zone, htg, hrg, pol, phi_t, phi_r, lam_t, lam_r = params
            
            results.append({
                'profile_id': idx + 1,
                'frequency_ghz': f,
                'time_percentage': p,
                'polarization': pol,
                'tx_lat': phi_t,
                'tx_lon': lam_t,
                'rx_lat': phi_r,
                'rx_lon': lam_r,
                'antenna_height_tx_m': htg,
                'antenna_height_rx_m': hrg,
                'max_distance_km': float(d[-1]),
                'num_distance_points': len(d),
                'Lb_dB': Lb,
                'Ep_dBuV_m': Ep,
            })
            
            if (idx + 1) % 10 == 0:
                print(f"  {idx + 1}/{len(profiles)}...")
        except Exception as e:
            failed += 1
    
    elapsed = time.perf_counter() - start_total
    print(f"\n✓ Complete: {len(results)}/{len(profiles)} succeeded in {elapsed:.2f}s")
    
    return pd.DataFrame(results)

# Process all
if profiles:
    print(f"Processing {len(profiles)} profiles...\n")
    results_df = process_all_profiles(profiles)
else:
    results_df = pd.DataFrame()

## 8. Results Summary

In [ ]:
if not results_df.empty:
    print(f"Shape: {results_df.shape}")
    print(f"\nFirst 5 rows:")
    print(results_df.head())
    print(f"\nStatistics:")
    print(results_df[['Lb_dB', 'Ep_dBuV_m']].describe())
else:
    print("No results")

## 9. Export to CSV

In [ ]:
if not results_df.empty:
    csv_path = output_dir / 'p1812_results.csv'
    results_df.to_csv(csv_path, index=False)
    print(f"✓ Exported to {csv_path}")
    print(f"  Size: {csv_path.stat().st_size / 1024:.1f} KB")

## 10. Visualizations

In [ ]:
if not results_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    ax1.hist(results_df['Lb_dB'].dropna(), bins=20, edgecolor='black', alpha=0.7)
    ax1.set_xlabel('Lb (dB)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Transmission Loss Distribution')
    ax1.grid(alpha=0.3)
    
    ax2.hist(results_df['Ep_dBuV_m'].dropna(), bins=20, edgecolor='black', alpha=0.7, color='orange')
    ax2.set_xlabel('Ep (dBμV/m)')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Electric Field Strength Distribution')
    ax2.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()